In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from pcdet.utils import common_utils
from pcdet.config import cfg, cfg_from_yaml_file
import pcdet.datasets.processor.data_processor as DP
import pcdet.datasets.dataset as D

# os.environ["CUDA_VISIBLE_DEVICES"] = "4"
# torch.cuda.set_device(4)

ckpt_path = './../output/kitti_models/voxel_rcnn_car/A_S22/ckpt/checkpoint_epoch_160.pth'
cfg_path = './cfgs/kitti_models/voxel_rcnn_car.yaml'
dataset_cfg = './cfgs/dataset_configs/kitti_dataset_copy.yaml'
data_path = '/home/jiazx_ug/dataset/S2/A/training/velodyne/128976.bin'
txt_path = '/home/jiazx_ug/dataset/S2/A/training/label_2/128976.txt'
data_dir_path = '/home/jiazx_ug/dataset/S4/temp/training/'


In [2]:
# use the parameter in the config file, voxelize the point cloud

# 1. load parameters
logger = common_utils.create_logger() 
cfg_from_yaml_file(cfg_path, cfg)
cfg_from_yaml_file(dataset_cfg, cfg.DATA_CONFIG)

class_names = cfg.CLASS_NAMES
point_cloud_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE
training = False
num_point_features = cfg.DATA_CONFIG.DATA_AUGMENTOR.AUG_CONFIG_LIST[0].NUM_POINT_FEATURES
dataset_processor_cfg = cfg.DATA_CONFIG.DATA_PROCESSOR
voxel_size = dataset_processor_cfg[2].VOXEL_SIZE
max_number_per_voxel = dataset_processor_cfg[2].MAX_POINTS_PER_VOXEL
points_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE


In [3]:
import numpy as np
from skimage import transform
import torch
import torchvision
import spconv
tv = None
try:
    import cumm.tensorview as tv
except:
    pass
class voxelgenerator():
    def __init__(self,vsize_xyz, coors_range_xyz, num_point_features, max_num_points_per_voxel, max_num_voxels):
        try:
            from spconv.utils import VoxelGeneratorV2 as VoxelGenerator
            self.spconv_ver = 1
        except:
            try:
                from spconv.utils import VoxelGenerator
                self.spconv_ver = 1
            except:
                from spconv.utils import Point2VoxelCPU3d as VoxelGenerator
                self.spconv_ver = 2

        if self.spconv_ver == 1:
            self._voxel_generator = VoxelGenerator(
                voxel_size=vsize_xyz,
                point_cloud_range=coors_range_xyz,
                max_num_points=max_num_points_per_voxel,
                max_voxels=max_num_voxels
            )
        else:
            self._voxel_generator = VoxelGenerator(
                vsize_xyz=vsize_xyz,
                coors_range_xyz=coors_range_xyz,
                num_point_features=num_point_features,
                max_num_points_per_voxel=max_num_points_per_voxel,
                max_num_voxels=max_num_voxels
            )

    def generate(self, points):
        if self.spconv_ver == 1:
            voxel_output = self._voxel_generator.generate(points)
            if isinstance(voxel_output, dict):
                voxels, coordinates, num_points = \
                    voxel_output['voxels'], voxel_output['coordinates'], voxel_output['num_points_per_voxel']
            else:
                voxels, coordinates, num_points = voxel_output
        else:
            assert tv is not None, f"Unexpected error, library: 'cumm' wasn't imported properly."
            voxel_output = self._voxel_generator.point_to_voxel(tv.from_numpy(points))
            tv_voxels, tv_coordinates, tv_num_points = voxel_output
            # make copy with numpy(), since numpy_view() will disappear as soon as the generator is deleted
            voxels = tv_voxels.numpy()
            coordinates = tv_coordinates.numpy()
            num_points = tv_num_points.numpy()
        return voxels, coordinates, num_points
    
    def meanVFE(self,voxels, num_points):
        # calculate the mean of each voxel
        

    

voxel_size = dataset_processor_cfg[2].VOXEL_SIZE
points_range = cfg.DATA_CONFIG.POINT_CLOUD_RANGE
num_point_features = cfg.DATA_CONFIG.DATA_AUGMENTOR.AUG_CONFIG_LIST[0].NUM_POINT_FEATURES
max_number_per_voxel = dataset_processor_cfg[2].MAX_POINTS_PER_VOXEL
max_number_of_voxels = dataset_processor_cfg[2].MAX_NUMBER_OF_VOXELS['train']

voxel_generator = voxelgenerator(voxel_size, points_range, num_point_features, max_number_per_voxel, max_number_of_voxels)
points = np.fromfile(data_path, dtype=np.float32).reshape(-1, 4)[:, :4]
voxels, coordinates, num_points = voxel_generator.generate(points)
print(voxels.shape, coordinates.shape, num_points.shape)
meanVFE = voxel_generator.meanVFE(voxels,num_points)


(16000, 5, 4) (16000, 3) (16000,)


TypeError: _sum() got an unexpected keyword argument 'dim'

In [ ]:
# from spconv import utils
# from spconv.utils import (non_max_suppression_cpu,
#                                  points_to_voxel_3d_np,
#                                  points_to_voxel_3d_np_mean,
#                                  points_to_voxel_3d_with_filtering,
#                                  rbbox_intersection, rbbox_iou,
#                                  rotate_non_max_suppression_cpu)
 
# try:
#     from spconv.utils import non_max_suppression
# except ImportError:
#     pass
 
 
# def points_to_voxel(points,
#                     voxel_size,
#                     coors_range,
#                     coor_to_voxelidx,
#                     max_points=35,
#                     max_voxels=20000,
#                     full_mean=False,
#                     block_filtering=True,
#                     block_factor=1,
#                     block_size=8,
#                     height_threshold=0.2,
#                     height_high_threshold=3.0,
#                     pad_output=False):
#     """convert 3d points(N, >=3) to voxels. This version calculate
#     everything in one loop. now it takes only 0.8ms(~6k voxels) 
#     with c++ and 3.2ghz cpu.
#     Args:
#         points: [N, ndim] float tensor. points[:, :3] contain xyz points and
#             points[:, 3:] contain other information such as reflectivity.
#         voxel_size: [3] list/tuple or array, float. xyz, indicate voxel size
#         coors_range: [6] list/tuple or array, float. indicate voxel range.
#             format: xyzxyz, minmax
#         coor_to_voxelidx: int array. used as a dense map.
#         max_points: int. indicate maximum points contained in a voxel.
#         max_voxels: int. indicate maximum voxels this function create.
#             for voxelnet, 20000 is a good choice. you should shuffle points
#             before call this function because max_voxels may drop some points.
#         full_mean: bool. if true, all empty points in voxel will be filled with mean
#             of exist points.
#         block_filtering: filter voxels by height. used for lidar point cloud.
#             use some visualization tool to see filtered result.
#     Returns:
#         voxels: [M, max_points, ndim] float tensor. only contain points.
#         coordinates: [M, 3] int32 tensor. zyx format.
#         num_points_per_voxel: [M] int32 tensor.
#     """
#     if full_mean:
#         assert block_filtering is False
#     if not isinstance(voxel_size, np.ndarray):
#         voxel_size = np.array(voxel_size, dtype=points.dtype)
#     if not isinstance(coors_range, np.ndarray):
#         coors_range = np.array(coors_range, dtype=points.dtype)
#     voxelmap_shape = (coors_range[3:] - coors_range[:3]) / voxel_size
#     voxelmap_shape = tuple(np.round(voxelmap_shape).astype(np.int32).tolist())
#     voxelmap_shape = voxelmap_shape[::-1]
#     num_points_per_voxel = np.zeros(shape=(max_voxels, ), dtype=np.int32)
#     voxels = np.zeros(shape=(max_voxels, max_points, points.shape[-1]),
#                       dtype=points.dtype)
#     voxel_point_mask = np.zeros(shape=(max_voxels, max_points),
#                                 dtype=points.dtype)
#     coors = np.zeros(shape=(max_voxels, 3), dtype=np.int32)
#     res = {
#         "voxels": voxels,
#         "coordinates": coors,
#         "num_points_per_voxel": num_points_per_voxel,
#         "voxel_point_mask": voxel_point_mask,
#     }
#     if full_mean:
#         means = np.zeros(shape=(max_voxels, points.shape[-1]),
#                          dtype=points.dtype)
#         voxel_num = points_to_voxel_3d_np_mean(points, voxels,
#                                                voxel_point_mask, means, coors,
#                                                num_points_per_voxel,
#                                                coor_to_voxelidx,
#                                                voxel_size.tolist(),
#                                                coors_range.tolist(),
#                                                max_points, max_voxels)
#     else:
#         if block_filtering:
#             block_shape = [*voxelmap_shape[1:]]
#             block_shape = [b // block_factor for b in block_shape]
#             mins = np.full(block_shape, 99999999, dtype=points.dtype)
#             maxs = np.full(block_shape, -99999999, dtype=points.dtype)
#             voxel_mask = np.zeros((max_voxels, ), dtype=np.int32)
#             voxel_num = points_to_voxel_3d_with_filtering(
#                 points, voxels, voxel_point_mask, voxel_mask, mins, maxs,
#                 coors, num_points_per_voxel, coor_to_voxelidx,
#                 voxel_size.tolist(), coors_range.tolist(), max_points,
#                 max_voxels, block_factor, block_size, height_threshold,
#                 height_high_threshold)
#             voxel_mask = voxel_mask.astype(np.bool_)
#             coors_ = coors[voxel_mask]
#             if pad_output:
#                 res["coordinates"][:voxel_num] = coors_
#                 res["voxels"][:voxel_num] = voxels[voxel_mask]
#                 res["voxel_point_mask"][:voxel_num] = voxel_point_mask[
#                     voxel_mask]
 
#                 res["num_points_per_voxel"][:voxel_num] = num_points_per_voxel[
#                     voxel_mask]
#                 res["coordinates"][voxel_num:] = 0
#                 res["voxels"][voxel_num:] = 0
#                 res["num_points_per_voxel"][voxel_num:] = 0
#                 res["voxel_point_mask"][voxel_num:] = 0
#             else:
#                 res["coordinates"] = coors_
#                 res["voxels"] = voxels[voxel_mask]
#                 res["num_points_per_voxel"] = num_points_per_voxel[voxel_mask]
#                 res["voxel_point_mask"] = voxel_point_mask[voxel_mask]
#             voxel_num = coors_.shape[0]
#         else:
#             voxel_num = points_to_voxel_3d_np(points, voxels, voxel_point_mask,
#                                               coors, num_points_per_voxel,
#                                               coor_to_voxelidx,
#                                               voxel_size.tolist(),
#                                               coors_range.tolist(), max_points,
#                                               max_voxels)
#     res["voxel_num"] = voxel_num
#     res["voxel_point_mask"] = res["voxel_point_mask"].reshape(
#         -1, max_points, 1)
#     return res
 
 
# class VoxelGenerator:
#     def __init__(self,
#                  voxel_size,
#                  point_cloud_range,
#                  max_num_points,
#                  max_voxels=20000,
#                  full_mean=True):
#         point_cloud_range = np.array(point_cloud_range, dtype=np.float32)
#         # [0, -40, -3, 70.4, 40, 1]
#         voxel_size = np.array(voxel_size, dtype=np.float32)
#         grid_size = (point_cloud_range[3:] -
#                      point_cloud_range[:3]) / voxel_size
#         grid_size = np.round(grid_size).astype(np.int64)
#         voxelmap_shape = tuple(np.round(grid_size).astype(np.int32).tolist())
#         voxelmap_shape = voxelmap_shape[::-1]
 
#         self._coor_to_voxelidx = np.full(voxelmap_shape, -1, dtype=np.int32)
#         self._voxel_size = voxel_size
#         self._point_cloud_range = point_cloud_range
#         self._max_num_points = max_num_points
#         self._max_voxels = max_voxels
#         self._grid_size = grid_size
#         self._full_mean = full_mean
 
#     def generate(self, points, max_voxels=None):
#         res = points_to_voxel(points, self._voxel_size,
#                               self._point_cloud_range, self._coor_to_voxelidx,
#                               self._max_num_points, max_voxels
#                               or self._max_voxels, self._full_mean)
#         voxels = res["voxels"]
#         coors = res["coordinates"]
#         num_points_per_voxel = res["num_points_per_voxel"]
#         voxel_num = res["voxel_num"]
#         coors = coors[:voxel_num]
#         voxels = voxels[:voxel_num]
#         num_points_per_voxel = num_points_per_voxel[:voxel_num]
 
#         return (voxels, coors, num_points_per_voxel)
 
#     def generate_multi_gpu(self, points, max_voxels=None):
#         res = points_to_voxel(points, self._voxel_size,
#                               self._point_cloud_range, self._coor_to_voxelidx,
#                               self._max_num_points, max_voxels
#                               or self._max_voxels, self._full_mean)
#         voxels = res["voxels"]
#         coors = res["coordinates"]
#         num_points_per_voxel = res["num_points_per_voxel"]
#         voxel_num = res["voxel_num"]
#         return (voxels, coors, num_points_per_voxel)
 
#     @property
#     def voxel_size(self):
#         return self._voxel_size
 
#     @property
#     def max_num_points_per_voxel(self):
#         return self._max_num_points
 
#     @property
#     def point_cloud_range(self):
#         return self._point_cloud_range
 
#     @property
#     def grid_size(self):
#         return self._grid_size

ImportError: cannot import name 'points_to_voxel_3d_np' from 'spconv.utils' (/home/jiazx_ug/anaconda3/envs/det_cu117/lib/python3.8/site-packages/spconv/utils/__init__.py)